In [11]:
import numpy as np
import os
import json
from datetime import datetime

In [12]:
# %cd ..

In [13]:
city_name = 'rotterdam'
config_path = f"""data/{city_name}/output_20241009_092045/config.json"""

In [14]:
with open(config_path, 'r') as file:
    CONFIG = json.load(file)
    print(CONFIG)


folder_path = CONFIG['study_area']['data_root']
data_root = os.path.join(folder_path, CONFIG['output_folder_name'])
point_grid_path = os.path.join(data_root, 'intermediate', 'point_grid.dat')

estimated_irradiance_path = os.path.join(data_root, 'irradiance.npy')
simplified_irradiance_path = os.path.join(data_root, 'simplified_irradiance.npy')
point_grid = np.loadtxt(point_grid_path)
estimated_irradiance = np.load(estimated_irradiance_path)
# simplified_irradiance = np.load(simplified_irradiance_path)

# simplified_no_global = estimated_irradiance[0] + simplified_irradiance[2]

{'area_per_point': 9, 'azimuth_resolution': 5, 'elevation_resolution': 5, 'epw_file': 'heino.epw', 'irradiance_batch_size': 500, 'irradiance_model': 'isotropic', 'num_bounces': 2, 'output_folder_name': 'output_20241005_122441', 'result': {'bbox_max': [235155.0, 484045.0, 54.15999984741211], 'bbox_min': [229055.0, 477845.0, 4.028999328613281], 'num_timesteps': 54, 'shadow_calculation_time': 1.0173659324645996, 'viewshed_rendering_time': 151.43592834472656, 'voxel_dim_x': 678, 'voxel_dim_y': 689, 'voxel_dim_z': 6, 'direct_diffuse_time': 5.968583106994629, 'global_irradiance_time': 42941.120844602585}, 'scenario': 'weatherstation', 'study_area': {'data_root': './data/RH_small', 'end_time': '2021-02-02 00:00:00', 'frequency': '10min', 'lat': 52.306205367362864, 'long': 6.51767410810534, 'start_time': '2021-02-01 00:00:00', 'timezone': 'Europe/Amsterdam'}, 'voxel_resolution': 9}


In [15]:
point_coords = point_grid[:,:3] # x,y,z, nx, ny, nz, albedo, surface_type, surface_id


In [16]:
bounce1_irradiance = estimated_irradiance[1]
bounce1_irradiance = np.round(bounce1_irradiance/6, decimals=3) # unit is Wh/m2


sum_irradiance = np.sum(bounce1_irradiance,axis=1).reshape(-1,1) # unit is Wh/m2
sum_irradiance = np.round(sum_irradiance, decimals=3)

# simplified_sum = np.sum(np.round(simplified_no_global/1000, decimals=3), axis=1).reshape(-1,1)
# simplified_sum = np.round(simplified_sum, decimals=3)


In [17]:
# point_grid_with_irradiance = np.hstack((point_coords,sum_irradiance, simplified_sum, bounce1_irradiance))
point_grid_with_irradiance = np.hstack((point_coords,sum_irradiance))

In [18]:
yeardaytime = CONFIG["study_area"]["start_time"]

In [19]:
dt = datetime.strptime(yeardaytime, '%Y-%m-%d %H:%M:%S')
new_date_string = dt.strftime('%Y-%m-%d')
result_path = f"""./data/figures/{city_name}_{new_date_string}_pc.txt"""

In [20]:
np.savetxt(result_path,point_grid_with_irradiance)